Import all required tools

In [17]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform("local")

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [18]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_tech",
    "introducing the add_tech feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

#scen.commit(comment="Introducing emissions and setting an upper bound")
#scen.set_as_default()
#scen.solve()
#scen.var("OBJ")["lvl"]

Call all data required by add_tech module

In [19]:
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# As if these parameters provided in the arguments
#regions = ['Westeros', 'Westerlands']
#modes = ['standard']
#emissions = ['CO2','CH4']
#time = ['year']

regions = []
modes = []
emissions = []
times = []
commodities = []
levels = []

first_active_year = tech_data['model_data'].get('first_active_year')
# If those are not provided, then this block of code is needed to retrieve them from the data input
if not regions:
    regions = list(tech_data.get('model_data',{}).get('DACCS',{}).get('fix_cost',{}).get('node_loc').keys())
if not emissions:
    emissions = list(tech_data.get('model_data',{}).get('DACCS',{}).get('emission_factor',{}).get('emission').keys())
if not modes:
    modes = list(tech_data.get('model_data',{}).get('DACCS',{}).get('var_cost',{}).get('mode').keys())
if not times:
    times = list(tech_data.get('model_data',{}).get('DACCS',{}).get('var_cost',{}).get('time').keys())
if not commodities:
    commodities = list(tech_data.get('model_data',{}).get('DACCS',{}).get('input',{}).get('commodity').keys())
if not levels:
    levels = list(tech_data.get('model_data',{}).get('DACCS',{}).get('input',{}).get('level').keys())

set_elements_dict = {
    'emission':emissions,
    'mode':modes,
    'time':times,
    'commodity':commodities,
    'level':levels,
    'time_origin':times,
    'time_dest':times,
}    

#list(scen.set('node'))

Define years vintage and active according to `first_active_year` and scenarios year

In [20]:
years_vtg_act = scen.vintage_and_active_years()
years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= first_active_year]

#years_vtg_act

Creating basic dataframe to be filled in later

In [21]:
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}


# Basic DataFrame
count = 0
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            # identify parameters by year dimension
            # then add the year data as kwargs as input for basic dataframe
            if all(e in parameters[par] for e in ['year_vtg','year_act']):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            elif 'year_vtg' in parameters[par]:
                kwargs = {'year_vtg': sorted(set(years_vtg_act['year_vtg']))}
            else:
                kwargs = {'year_act': sorted(set(years_vtg_act['year_act']))}
            
            # create parameter's basic dataframe and 
            # add it to the data parameter list
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                ))

            # duplicate the basic data using the length of each set
            # as the duplication factor
            for s, elist in set_elements_dict.items():
                if s in parameters[par]:
                    data[par] = data[par]*len(elist)
                    for e in range(len(elist)):
                        kwarg = {s:elist[e]}
                        data[par][e] = data[par][e].assign(**kwarg)
                data[par] = [pd.concat(data[par]).reset_index(drop=True)]

data = {k: pd.concat(v).reset_index(drop=True) for k, v in data.items()}

In [22]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    for tech, diffs in tech_data['model_data'].items():
        if tech != 'first_active_year':
            for reg in regions:
                multiplier = []
                for i in range(len(data[par])):
                    # Calculate multipliers for each element in a dimensional array.
                    # For each element, this function searches for corresponding factors
                    # in the model-specific data (model_data).
                    # If no factors are found, the multiplier is set to 1.
                    # If factors are found, the function uses the factor that matches
                    # the corresponding element in the data[par] row.
                    
                    # get regional multiplier from model_data
                    m_reg = diffs.get(par,{}).get('node_loc',{}).get(reg,1)
                    
                    # get year_vtg escalation rate from model_data
                    # then calculate year_vtg multiplier
                    # m_year_vtg = (1+rate)**delta_years
                    m_year_vtg = (((1+diffs.get(par,{}).get('year_vtg',{}).get('rate',0)) 
                                  **(data[par]['year_vtg'][i]-first_active_year)) 
                                  if 'year_vtg' in parameters[par] else 1)
                    
                    # same as m_year_vtg
                    # m_year_act = (1+rate)**(year_act-year_vtg) if both years present
                    # m_year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
                    m_year_act = (((1+diffs.get(par,{}).get('year_act',{}).get('rate',0))
                                  **(data.get(par,{}).get('year_act',{})
                                     .get(i,0)
                                     -data[par]['year_vtg'][i] if 'year_vtg' in parameters[par] else first_active_year))
                                 if 'year_act' in parameters[par] else 1)
                    
                    # get mode multiplier from model_data
                    m_mode = (diffs.get(par,{}).get('mode',{})
                              .get(data.get(par,{}).get('mode',{}).get(i),1))
                    
                    # get emission multiplier from model_data
                    m_emission = (diffs.get(par,{}).get('emission',{})
                                  .get(data.get(par,{}).get('emission',{}).get(i),1))
                    
                    multiplier.append(
                            np.prod([m_reg, m_year_vtg, m_year_act, m_mode, m_emission,])
                        )
                    
                value = data[par]['value']*multiplier
                
                # node_origin and node_dest are assumed to be always the same of the node
                kwargs = {}
                if 'node_origin' in parameters[par]:
                    kwargs.update({'node_origin': reg})
                if 'node_dest' in parameters[par]:
                    kwargs.update({'node_dest': reg})
                
                # assigning data expansion
                data_expand[par].append(
                    data[par].assign(node_loc=reg,value=value, **kwargs)
                   )

In [23]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in parameters.keys()}

In [24]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [25]:
mp.close_db()